In [1]:
from numpy import *
from scipy.special import jn, yn, jvp, yvp, jn_zeros
from scipy.constants import c, epsilon_0, mu_0
from scipy.optimize import brentq

In [2]:
def f0(R0):
    # unperturbed frequency, Pozar pp 290, 
    # TM(n, m, l): f = c/(2*pi) * sqrt( (p_nm/a)**2 + (l*pi/d)**2)
    # p_nm - m-th root of J_n
    p_01 = jn_zeros(0, 1)[0]
    f = c/(2*pi) * p_01/R0
    return f

In [3]:
def f_perturbed(R0, R1, eps_r):
    # simple perturbation theory for solid rod inserted, Li, Akyel, Bosisio;
    # http://dx.doi.org/10.1109/TMTT.1981.1130496
    # eps_p - 1 = 2*J_1^2(x_01)*df/f * R0^2/R1^2
    p_01 = jn_zeros(0, 1)[0]
    df_rel = (eps_r-1) / (2*jn(1, p_01)**2) * R1**2/R0**2
    f = f0(R0)*(1-df_rel)
    return f

In [4]:
def f_Li(R0, R1, R2, eps_r, eps_g):
    # Li, Lukac theory:
    # http://dx.doi.org/10.1109/TMTT.1981.1130496
    eps_1 = eps_r
    eps_2 = eps_g
    def eq(fs):
        k0 = 2*pi*fs/c
        k = [k0, sqrt(eps_1)*k0, sqrt(eps_2)*k0]
        R = [R0, R1, R2]
        def J(n, p, q): return jn(n, k[p]*R[q])
        def Y(n, p, q): return yn(n, k[p]*R[q])
        A = sqrt(eps_2) * (J(0,0,0)*Y(0,0,2) - J(0,0,2)*Y(0,0,0)) / (J(0,0,0)*Y(1,0,2) - J(1,0,2)*Y(0,0,0))
        nom = J(1,2,1)*Y(0,2,2) - Y(1,2,1)*J(0,2,2) - A*(J(1,2,1)*Y(1,2,2) - Y(1,2,1)*J(1,2,2))
        den = J(0,2,1)*Y(0,2,2) - Y(0,2,1)*J(0,2,2) - A*(J(0,2,1)*Y(1,2,2) - Y(0,2,1)*J(1,2,2))
        F = sqrt(eps_2)*k0*nom/den
        lhs = k[1]*J(1,1,1)/J(0,1,1)
        return lhs-F
    
    #print(eq(f0(R0)*0.5), eq(f0(R0)*1.2))
    f = brentq(eq, f0(R0)*0.3, f0(R0)*1.2)
    return f


In [5]:
def printcols(c1, c2):
        print("%s%s" % (c1.ljust(45),c2))
def print_params(R, f1, f2, R1=None, R2=None):
    if R is not None:
        printcols("Radius of empty cavity for 2.45 GHz", "r = %.2f mm" % (R*1e3))
    if R1 is not None:
        printcols("Sapphire inner radius", "R1 = %.2f mm" % (R1*1e3))
    if R2 is not None:
        printcols("Sapphire outer radius", "R2 = %.2f mm" % (R2*1e3))
    if f1 is not None:
        printcols("Unperturbed frequency","f = %.3f GHz" % (f1/1e9))
    if isinstance(f2, tuple):
        printcols("Frequency with tube", "f = %.3f -- %.3f GHz" % (f2[0]/1e9, f2[1]/1e9))
    else:
        printcols("Frequency with tube", "f = %.3f GHz" % (f2/1e9,))
    

In [6]:
# dielectric sample inserted in TM010 cavity
# n, m, l - angular, radial, axial

f_target = 2.45e9
f_measured = 2.52e9

f_hole = 26e6 # calculated correction for presence of holes in the resonator

eps_r = 1     # vacuum relative perimittivity

### Empty cavity for 2.45 GHz

In [7]:
# calculate optimal cavity radius for targed freq
R0 = brentq(lambda x: f0(x)-f_target, 30e-3, 70e-3)
R1 = 11.2e-3
R2 = 12.6e-3 # containing tube radius
eps_g = 10

print_params(R0, f0(R0), f_Li(R0, R1, R2, eps_r, eps_g))

Radius of empty cavity for 2.45 GHz          r = 46.83 mm
Unperturbed frequency                        f = 2.450 GHz
Frequency with tube                          f = 2.029 GHz


### Test cavity initial design

In [8]:
R0 = 36.e-3
R1 = 11.e-3
R2 = 12.5e-3 # containing tube radius
eps_g = 10

print_params(R0, f0(R0), f_Li(R0, R1, R2, eps_r, eps_g))

Radius of empty cavity for 2.45 GHz          r = 36.00 mm
Unperturbed frequency                        f = 3.187 GHz
Frequency with tube                          f = 2.437 GHz


### More realistic calculation of current resonator frequency

More accurate dimensions of the used tube, accounting for the holes in the resonator and for parallel and perpendicular orientations of the sapphire optical axis w.r.t. the tube axis.

In [9]:
R0 = 36e-3
R2 = (25.05+25.25+25.10+25.30)/8*1e-3   # diameter measurements
R1 = R2 - (1.2+1.4)/2*1e-3              # thickness measurements
eps_g_perp = 9.3                        # average permittivity perpendicular to optical axis
eps_g_para = 11.5                       # permittivity along optical axis
                                        # http://www.crytur.cz/materials/sapphire/
                                        # eps = 11.5 parallel to c axis
                                        # eps = 9.3 perpendicular to c axis

print_params(None, f_hole + f0(R0), (f_hole + f_Li(R0, R1, R2, eps_r, eps_g_para),
    f_hole + f_Li(R0, R1, R2, eps_r, eps_g_perp)), R1=R1, R2=R2)

Sapphire inner radius                        R1 = 11.29 mm
Sapphire outer radius                        R2 = 12.59 mm
Unperturbed frequency                        f = 3.213 GHz
Frequency with tube                          f = 2.455 -- 2.573 GHz


### Test cavity - actual sapphire permittivity?

In [10]:
eps_g = brentq(lambda x: f_hole + f_Li(R0, R1, R2, eps_r, x)-f_measured, 5, 15)
printcols("Sapphire permittivity to match f_measured", "eps_g = %f" % (eps_g))
print_params(None, f0(R0), f_hole + f_Li(R0, R1, R2, eps_r, eps_g))

R0 = brentq(lambda x: f_hole + f_Li(x, R1, R2, eps_r, eps_g)-f_target, 30e-3, 50e-3)
print()
printcols("Corrected cavity radius to match f_target", "R0 = %f mm" % (R0*1e3))
print_params(None, f0(R0), f_hole+f_Li(R0, R1, R2, eps_r, eps_g))

Sapphire permittivity to match f_measured    eps_g = 10.253863
Unperturbed frequency                        f = 3.187 GHz
Frequency with tube                          f = 2.520 GHz

Corrected cavity radius to match f_target    R0 = 37.378996 mm
Unperturbed frequency                        f = 3.070 GHz
Frequency with tube                          f = 2.450 GHz


### Test cavity - actual sapphire outer radius assuming parallel orientation?

In [11]:
R0 = 36e-3
eps_g = eps_g_para
R2 = brentq(lambda x: f_Li(R0, R1, x, eps_r, eps_g)-f_measured, R1, R1*1.5)
print_params(None, f0(R0), f_Li(R0, R1, R2, eps_r, eps_g), R2=R2)

print()
R0 = brentq(lambda x: f_Li(x, R1, R2, eps_r, eps_g)-f_target, 30e-3, 50e-3)
printcols("Corrected cavity radius to match f_target", "R0 = %f mm" % (R0*1e3))
print_params(None, f0(R0), f_Li(R0, R1, R2, eps_r, eps_g))

Sapphire outer radius                        R2 = 12.38 mm
Unperturbed frequency                        f = 3.187 GHz
Frequency with tube                          f = 2.520 GHz

Corrected cavity radius to match f_target    R0 = 37.354852 mm
Unperturbed frequency                        f = 3.072 GHz
Frequency with tube                          f = 2.450 GHz


## Corrected resonator dimensions

I propose to use 36.8 mm as the final resonator cavity diameter. In this case the theoretical frequency range is:

In [12]:
R0_new = 36.8e-3
R2 = (25.05+25.25+25.10+25.30)/8*1e-3 # diameter measurements
R1 = R2 - (1.2+1.4)/2*1e-3            # thickness measurements

print_params(None, None, (f_hole + f_Li(R0_new, R1, R2, eps_r, eps_g_para),
    f_hole + f_Li(R0_new, R1, R2, eps_r, eps_g_perp)))

Frequency with tube                          f = 2.415 -- 2.531 GHz


Taking into account manufacturing tolerances 0.2 mm, and thermal contraction of aluminum of 0.4 % at cryogenic temperatures compared to room temperature:

In [13]:
R0_min = (R0_new - 0.2e-3) * (1 - 4e-4)
R0_max = R0_new + .2e-3
print_params(None, None, (f_hole + f_Li(R0_max, R1, R2, eps_r, eps_g_para),
    f_hole + f_Li(R0_min, R1, R2, eps_r, eps_g_perp)))

Frequency with tube                          f = 2.406 -- 2.542 GHz


The measured frequency will be shifted to (blaming the sapphire permittivity... :):

In [14]:
R0 = 36e-3
eps_g = brentq(lambda x: f_hole + f_Li(R0, R1, R2, eps_r, x)-f_measured, 5, 15)
print_params(None, None, f_hole + f_Li(R0_new, R1, R2, eps_r, eps_g))

Frequency with tube                          f = 2.479 GHz


and it should lie in the range:

In [15]:
print_params(None, None, (f_hole + f_Li(R0_max, R1, R2, eps_r, eps_g),
    f_hole + f_Li(R0_min, R1, R2, eps_r, eps_g)))

Frequency with tube                          f = 2.469 -- 2.490 GHz


which is within the range of our synthesizer and amplifier